# 📦 CMake Seviye 3 – Kütüphane Oluşturma ve Linkleme Mantığı

Bu bölümde CMake'in en kritik seviyelerinden birine geçiyoruz:  
**Bir kütüphane oluşturmak ve bu kütüphaneyi projeye bağlamak.**

Bu mekanizmayı öğrendiğinde, daha sonraki aşamalarda:

- `find_package(OpenCV)`
- `find_package(Torch)`
- `${OpenCV_LIBS}`
- `${TORCH_LIBRARIES}`

gibi dış kütüphaneleri projeye nasıl dahil edeceğini **tam olarak** kavramış olacaksın.

---

## 🎯 Amaç

Bu mini proje ile:

1. Kendi kütüphanemizi (`my_math_lib`) oluşturacağız.  
2. Bu kütüphaneyi `add_library()` ile derleyeceğiz.  
3. `add_subdirectory()` ile ana projeye dahil edeceğiz.  
4. `target_link_libraries()` ile executable’a bağlayacağız.  
5. Include dizinlerini CMake’e doğru şekilde tanıtacağız.

Bu yapı doğrudan **OpenCV + LibTorch linking modelinin** minyatür halidir.

---

# 📁 Proje Yapısı

Oluşturacağımız proje:

```bash
Proje_3/
├── CMakeLists.txt
├── src/
│ └── main.cpp
└── lib/
├── CMakeLists.txt
├── my_math.cpp
└── my_math.h
```


---

# 🔧 1) Kütüphane Kodu (`lib/my_math.h`)

```cpp
#pragma once

int uc_kati(int x);
```
# 🔧 2) Kütüphane Implementasyonu (lib/my_math.cpp)
```cpp
#include "my_math.h"

int uc_kati(int x)
{
    return x * 3;
}
```

# 🔧 3) Kullanıcı Programı (src/main.cpp)

```cpp
#include <iostream>
#include "my_math.h"

int main()
{
    int x = 4;
    std::cout << "x = " << x << "\\n";
    std::cout << "uc_kati(x) = " << uc_kati(x) << "\\n";
    return 0;
}
```

# 🧠 4) Kütüphane için CMake (lib/CMakeLists.txt)

Bu dosya kütüphaneyi tanımlar:
```cpp
add_library(my_math_lib
    my_math.cpp
)

target_include_directories(my_math_lib
    PUBLIC
        ${CMAKE_CURRENT_SOURCE_DIR}
)
```


Burası kritik:

* add_library → .cpp dosyasından bir kütüphane oluşturur.

* PUBLIC include directory → Bu kütüphane kullanıldığında header dosyalarını görebilsin diye.



# 🔧 5) Ana CMake Dosyası (CMakeLists.txt)

Bu dosya projeyi yönetir:
```cpp
cmake_minimum_required(VERSION 3.16)

project(Proje_3 LANGUAGES CXX)

set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED ON)
set(CMAKE_CXX_EXTENSIONS OFF)

# lib klasörünü projeye dahil et
add_subdirectory(lib)

# executable oluştur
add_executable(Proje_3
    src/main.cpp
)

# Header dizinleri tanıt
target_include_directories(Proje_3
    PRIVATE
        ${CMAKE_SOURCE_DIR}/lib
)

# Kütüphaneyi bağla
target_link_libraries(Proje_3
    PRIVATE
        my_math_lib
)

```
**Bu üç satır, dış kütüphane bağlamanın temel prensibidir:**
```cpp
add_subdirectory(lib)
target_include_directories(...)
target_link_libraries(...)
```


**Aynı mekanizma ileride şöyle olacak:**
```cpp
find_package(OpenCV REQUIRED)
target_link_libraries(app PRIVATE ${OpenCV_LIBS})

find_package(Torch REQUIRED)
target_link_libraries(app PRIVATE ${TORCH_LIBRARIES})
```

**Yani OpenCV / LibTorch eklemek aslında sadece bir satır farkı.**

# 🏗️ 6) Derleme

Terminalde:
```bash
mkdir build
cd build
cmake ..
cmake --build .
```


Derleme başarılıysa:

>./Proje_3


Beklenen çıktı:
```bash
x = 4
uc_kati(x) = 12